In [1]:
import os

api_key = os.getenv("DEEP_SEEK_API_KEY")

In [2]:
# client = OpenAI(api_key=api_key, base_url="https://api.deepseek.com")

# # Round 1
# messages = [{"role": "user", "content": "9.11 and 9.8, which is greater?"}]
# response = client.chat.completions.create(
#     model="deepseek-reasoner",
#     messages=messages
# )
# print(response)

# reasoning_content = response.choices[0].message.reasoning_content
# content = response.choices[0].message.content

# # Round 2
# messages.append({'role': 'assistant', 'content': content})
# messages.append({'role': 'user', 'content': "How many Rs are there in the word 'strawberry'?"})
# response = client.chat.completions.create(
#     model="deepseek-reasoner",
#     messages=messages
# )

In [ ]:
import autogen

config_list_deepseek_reasoner = [
    {
        "model": "deepseek-reasoner",
        "base_url": "https://api.deepseek.com/v1",
        "api_key": api_key,
        "api_type": "deepseek",
        "tags": ["deepseek"],
    }
]
config_list_openai_4o_mini = [
    {
        "model": "gpt-4o-mini",
        "api_key": os.getenv("OPENAI_API_KEY"),
    }
]
user_proxy = autogen.UserProxyAgent(
    "user_proxy",
    human_input_mode="NEVER",
    code_execution_config={"work_dir": "coding", "use_docker": False},
)

supervisor = autogen.AssistantAgent(
    "supervisor",
    llm_config={
        "config_list": config_list_deepseek_reasoner,
    },
)


assistant = autogen.AssistantAgent(
    "assistant",
    llm_config={
        "config_list": config_list_deepseek_reasoner,
    },
)


groupchat = autogen.GroupChat(
    agents=[user_proxy, supervisor, assistant],
    messages=["A group chat"],
    max_round=5,
)

manager = autogen.GroupChatManager(
    groupchat=groupchat,
    llm_config={
        # Error is expected using deepseek_reasoner as manager
        # "config_list": config_list_deepseek_reasoner,
        #############
        # raise self._make_status_error_from_response(err.response) from None
        # openai.BadRequestError: Error code: 400 -
        # {'error': {'message': 'The last message of deepseek-reasoner must be a user message,
        # or an assistant message with prefix mode on (refer to https://api-docs.deepseek.com/guides/chat_prefix_completion).',
        # 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_request_error'}}
        #############
        # Another expected using openai_4o_mini as manager
        "config_list": config_list_openai_4o_mini,
        #############
        # Error code: 400 - {'error': {'message': 'deepseek-reasoner does not support successive
        # user or assistant messages (messages[1] and messages[2] in your input).
        # You should interleave the user/assistant messages in the message sequence.',
        # 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_request_error'}}
        #############
    },
)


# Group Chat
# user_proxy.initiate_chat(
#     manager,
#     message="""

#     Save a chart of NVDA and TESLA stock price change YTD.

#     Start the chat with assistant.

#     And ask supervisor to check the work from assistant.

#     """,
# )

In [ ]:
import autogen
from autogen.coding import LocalCommandLineCodeExecutor


class DeepSeekReasonerAssistant(autogen.AssistantAgent):
    def __init__(self, name, **kwargs):
        super().__init__(name, **kwargs)

    def generate_reply(self, messages=None, sender=None, config=None):
        print(f"\n=== {self.name} Processing Messages ===")
        if messages is None:
            messages = self._oai_messages[sender]
        print(f"Messages: {messages}")

        # Get messages from chat history if available
        if hasattr(self, "chat_messages") and sender in self.chat_messages:
            chat_history = self.chat_messages[sender]
        else:
            chat_history = messages if messages else []
        print(f"Chat History: {chat_history}")

        if chat_history:
            # Create new message sequence
            new_messages = []

            # Add system message if it exists
            if hasattr(self, "system_message"):
                new_messages.append({"role": "system", "content": self.system_message})

            # Add original messages ensuring alternation
            for msg in chat_history:
                if len(new_messages) == 0 or msg["role"] != new_messages[-1]["role"]:
                    new_messages.append(msg)

            # Check if last message was from assistant
            if new_messages and new_messages[-1]["role"] == "assistant":
                new_messages.append({"role": "user", "content": "continue"})
            print(f"New Messages: {new_messages}")

            return super().generate_reply(messages=new_messages, sender=sender, config=config)
        return super().generate_reply(messages=messages, sender=sender, config=config)


# Create agents using the custom class
assistant = DeepSeekReasonerAssistant(
    "assistant",
    system_message="You are a coding assistant. Write code for the given task.",
    llm_config={
        "config_list": config_list_deepseek_reasoner,
    },
)

supervisor = DeepSeekReasonerAssistant(
    "supervisor",
    system_message="""You are a code reviewer. Review code for:
    1. Correctness
    2. Error handling
    3. Performance
    4. Best practices""",
    llm_config={
        "config_list": config_list_deepseek_reasoner,
    },
)


user_proxy = autogen.UserProxyAgent(
    "user_proxy",
    human_input_mode="NEVER",
    code_execution_config={
        "executor": LocalCommandLineCodeExecutor(work_dir="coding"),
    },
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
)

# Create group chat
groupchat = autogen.GroupChat(agents=[user_proxy, assistant, supervisor], messages=[], max_round=5)

# Manager with chat model
manager = autogen.GroupChatManager(
    groupchat=groupchat,
    system_message="""You are the coordinator. Follow this sequence:
    1. When user asks a question, select assistant to write code
    2. After assistant provides code, select supervisor to review it
    3. If the problem has been resolved respond with TERMINATE""",
    llm_config={
        "config_list": config_list_openai_4o_mini,
    },
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
)

print("Starting chat...")
user_proxy.initiate_chat(manager, message="Create a chart showing NVDA and TESLA stock prices.")

In [ ]:
[
    {"content": "Create a chart showing NVDA and TESLA stock prices.", "name": "user_proxy", "role": "user"},
    {
        "content": "To create a chart comparing NVDA and TESLA stock prices, you can use Python with libraries like `yfinance` (to fetch stock data) and `matplotlib` (for plotting). Here's a complete example:\n\n```python\n# Install required libraries if needed\n# !pip install yfinance matplotlib\n\nimport yfinance as yf\nimport matplotlib.pyplot as plt\n\n# Define the tickers and time period\ntickers = ['NVDA', 'TSLA']\nstart_date = '2023-01-01'\nend_date = '2024-01-01'\n\n# Download historical data\ndata = yf.download(tickers, start=start_date, end=end_date)['Adj Close']\n\n# Create plot\nplt.figure(figsize=(12, 6))\nplt.plot(data['NVDA'], label='NVIDIA (NVDA)', color='#76b900')  # NVIDIA green\nplt.plot(data['TSLA'], label='Tesla (TSLA)', color='#cc0000')    # Tesla red\n\n# Formatting\nplt.title('NVDA vs TESLA Stock Prices (2023)', fontsize=14)\nplt.xlabel('Date', fontsize=12)\nplt.ylabel('Adjusted Close Price (USD)', fontsize=12)\nplt.legend()\nplt.grid(True, which='both', linestyle='--', linewidth=0.5)\nplt.xticks(rotation=45)\nplt.tight_layout()\n\n# Show plot\nplt.show()\n```\n\nThis code will:\n1. Fetch adjusted closing prices for both stocks from Yahoo Finance\n2. Create a line chart comparing their performance\n3. Use official brand colors (NVIDIA green and Tesla red)\n4. Include proper labels and formatting\n\nThe chart will show the stock price trends from January 2023 to January 2024. You can modify the `start_date` and `end_date` variables to adjust the time period.\n\nMake sure you have an internet connection as the script needs to download stock data from Yahoo Finance. The first time you run it, you might need to install the required libraries using pip.",
        "name": "assistant",
        "role": "user",
    },
    {
        "content": 'exitcode: 1 (execution failed)\nCode output: \n[                       0%                       ]\n[*********************100%***********************]  2 of 2 completed\nTraceback (most recent call last):\n  File "/Users/robert/projects/ag2/.venv/lib/python3.10/site-packages/pandas/core/indexes/base.py", line 3805, in get_loc\n    return self._engine.get_loc(casted_key)\n  File "index.pyx", line 167, in pandas._libs.index.IndexEngine.get_loc\n  File "index.pyx", line 196, in pandas._libs.index.IndexEngine.get_loc\n  File "pandas/_libs/hashtable_class_helper.pxi", line 7081, in pandas._libs.hashtable.PyObjectHashTable.get_item\n  File "pandas/_libs/hashtable_class_helper.pxi", line 7089, in pandas._libs.hashtable.PyObjectHashTable.get_item\nKeyError: \'Adj Close\'\n\nThe above exception was the direct cause of the following exception:\n\nTraceback (most recent call last):\n  File "/Users/robert/projects/ag2/notebook/coding/Install required libraries if needed", line 13, in <module>\n    data = yf.download(tickers, start=start_date, end=end_date)[\'Adj Close\']\n  File "/Users/robert/projects/ag2/.venv/lib/python3.10/site-packages/pandas/core/frame.py", line 4101, in __getitem__\n    return self._getitem_multilevel(key)\n  File "/Users/robert/projects/ag2/.venv/lib/python3.10/site-packages/pandas/core/frame.py", line 4159, in _getitem_multilevel\n    loc = self.columns.get_loc(key)\n  File "/Users/robert/projects/ag2/.venv/lib/python3.10/site-packages/pandas/core/indexes/multi.py", line 3040, in get_loc\n    loc = self._get_level_indexer(key, level=0)\n  File "/Users/robert/projects/ag2/.venv/lib/python3.10/site-packages/pandas/core/indexes/multi.py", line 3391, in _get_level_indexer\n    idx = self._get_loc_single_level_index(level_index, key)\n  File "/Users/robert/projects/ag2/.venv/lib/python3.10/site-packages/pandas/core/indexes/multi.py", line 2980, in _get_loc_single_level_index\n    return level_index.get_loc(key)\n  File "/Users/robert/projects/ag2/.venv/lib/python3.10/site-packages/pandas/core/indexes/base.py", line 3812, in get_loc\n    raise KeyError(key) from err\nKeyError: \'Adj Close\'\n',
        "name": "user_proxy",
        "role": "user",
    },
]

In [ ]:
[
    {"content": "Create a chart showing NVDA and TESLA stock prices.", "name": "user_proxy", "role": "user"},
    {
        "content": "To create a chart comparing NVDA and TESLA stock prices, we'll use Python with the `yfinance` library to fetch stock data and `matplotlib` for plotting. Here's the code:\n\n```python\n# First install required libraries if you don't have them\n# !pip install yfinance matplotlib pandas\n\nimport yfinance as yf\nimport matplotlib.pyplot as plt\nimport matplotlib.dates as mdates\nimport pandas as pd\n\n# Download historical stock data for NVDA and TSLA (Tesla)\nnvda = yf.download('NVDA', period='1y')\ntsla = yf.download('TSLA', period='1y')\n\n# Create figure and axis\nplt.figure(figsize=(12, 6))\n\n# Plot closing prices\nplt.plot(nvda.index, nvda['Close'], label='NVDA', color='blue')\nplt.plot(tsla.index, tsla['Close'], label='TESLA', color='red')\n\n# Formatting\nplt.title('NVDA vs TESLA Stock Prices (1 Year)', fontsize=14)\nplt.xlabel('Date', fontsize=12)\nplt.ylabel('Price (USD)', fontsize=12)\nplt.legend()\nplt.grid(True, which='both', linestyle='--', alpha=0.5)\n\n# Format x-axis dates\nplt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))\nplt.gca().xaxis.set_major_locator(mdates.MonthLocator())\nplt.gcf().autofmt_xdate()\n\n# Show plot\nplt.tight_layout()\nplt.show()\n```\n\nThis code will:\n1. Download 1-year historical data for both stocks\n2. Create a comparison chart with:\n   - Different colors for each stock\n   - Proper date formatting on the x-axis\n   - Grid lines for readability\n   - USD price scale on the y-axis\n   - Legend and labels\n\nNote: The first time you run this, it may take a moment to download the historical data. The prices shown will be the daily closing prices adjusted for splits and dividends.\n\nFor a live version or different time period, you can modify the `period` parameter in the `yf.download()` calls (options: '1d', '5d', '1mo', '3mo', '6mo', '1y', '5y', 'max').",
        "name": "assistant",
        "role": "user",
    },
    {
        "content": "exitcode: 0 (execution succeeded)\nCode output: \n[*********************100%***********************]  1 of 1 completed\n\n[*********************100%***********************]  1 of 1 completed\nFigure(1200x600)\n",
        "name": "user_proxy",
        "role": "user",
    },
]

In [ ]:
[
    {"content": "Create a chart showing NVDA and TESLA stock prices.", "name": "user_proxy", "role": "user"},
    {
        "content": "To create a chart comparing NVDA and TESLA stock prices, we'll use Python with the `yfinance` library to fetch stock data and `matplotlib` for plotting. Here's the code:\n\n```python\n# First install required libraries if you don't have them\n# !pip install yfinance matplotlib pandas\n\nimport yfinance as yf\nimport matplotlib.pyplot as plt\nimport matplotlib.dates as mdates\nimport pandas as pd\n\n# Download historical stock data for NVDA and TSLA (Tesla)\nnvda = yf.download('NVDA', period='1y')\ntsla = yf.download('TSLA', period='1y')\n\n# Create figure and axis\nplt.figure(figsize=(12, 6))\n\n# Plot closing prices\nplt.plot(nvda.index, nvda['Close'], label='NVDA', color='blue')\nplt.plot(tsla.index, tsla['Close'], label='TESLA', color='red')\n\n# Formatting\nplt.title('NVDA vs TESLA Stock Prices (1 Year)', fontsize=14)\nplt.xlabel('Date', fontsize=12)\nplt.ylabel('Price (USD)', fontsize=12)\nplt.legend()\nplt.grid(True, which='both', linestyle='--', alpha=0.5)\n\n# Format x-axis dates\nplt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))\nplt.gca().xaxis.set_major_locator(mdates.MonthLocator())\nplt.gcf().autofmt_xdate()\n\n# Show plot\nplt.tight_layout()\nplt.show()\n```\n\nThis code will:\n1. Download 1-year historical data for both stocks\n2. Create a comparison chart with:\n   - Different colors for each stock\n   - Proper date formatting on the x-axis\n   - Grid lines for readability\n   - USD price scale on the y-axis\n   - Legend and labels\n\nNote: The first time you run this, it may take a moment to download the historical data. The prices shown will be the daily closing prices adjusted for splits and dividends.\n\nFor a live version or different time period, you can modify the `period` parameter in the `yf.download()` calls (options: '1d', '5d', '1mo', '3mo', '6mo', '1y', '5y', 'max').",
        "name": "assistant",
        "role": "user",
    },
    {
        "content": "exitcode: 0 (execution succeeded)\nCode output: \n[*********************100%***********************]  1 of 1 completed\n\n[*********************100%***********************]  1 of 1 completed\nFigure(1200x600)\n",
        "name": "user_proxy",
        "role": "user",
    },
]

In [ ]:
[
    {"role": "system", "content": "You are a coding assistant. Write code for the given task."},
    {"content": "Create a chart showing NVDA and TESLA stock prices.", "name": "user_proxy", "role": "user"},
]

In [ ]:
user_proxy = autogen.UserProxyAgent(
    "user_proxy",
    human_input_mode="NEVER",
)
assistant = autogen.AssistantAgent(
    "assistant",
    llm_config={
        "config_list": config_list_deepseek_reasoner,
    },
)

user_proxy.initiate_chat(
    assistant,
    message="How are you?",
)